In [1]:
import ray


In [2]:
ray.init(num_cpus=16)

2023-06-17 02:49:05,739	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.8.16
Ray version:,2.2.0


In [3]:
# ray.shutdown()

In [4]:
import pandas as pd

import logging
from bds.meel import approx_mc2, approx_mc2_core
from bds.bb import get_ground_truth_count
from bds.rule import Rule
from bds.utils import bin_random, randints
from logzero import logger

import numpy as np
from contexttimer import Timer

logger.setLevel(logging.WARN)

num_pts = 1000

ub = 0.8
lmbd = .1

delta = .8
eps = .8

show_progres = True
rand_seed = 1234

OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [5]:
n_reps = 1
num_rules_list = list(reversed([50, 100, 150, 200]))
# num_rules_list = list(reversed([200]))


np.random.seed(rand_seed)
res_rows = []
for _ in range(n_reps):
    for num_rules in num_rules_list:
        rule_random_seeds = randints(num_rules)
        rules = [
            Rule.random(i+1, num_pts, random_seed=rule_random_seeds[i])
            for i in range(num_rules)
        ]
        y = bin_random(num_pts)

        with Timer() as cbb_timer:
            test_cnt = approx_mc2(
                rules, y, lmbd=lmbd, ub=ub,
                delta=delta, eps=eps, rand_seed=rand_seed, show_progress=show_progres,
                parallel=True, log_level=logging.WARN
            )
        test_elapsed = cbb_timer.elapsed
        
        with Timer() as bb_timer:
            ref_count = get_ground_truth_count(rules, y, lmbd, ub)
        ref_elapsed = bb_timer.elapsed        

        res_rows.append((num_rules, test_elapsed, ref_elapsed, test_cnt, ref_count))

  0%|                                                                                                                               | 0/16 [00:00<?, ?it/s](pid=32822) OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(pid=32814) OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(pid=32816) OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(pid=32815) OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(pid=32818) OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(pid=32813) OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(pid=32819) OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(pid=32820) OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_

(approx_mc2_core_wrapper pid=32828) time_cost_info: [(1, 0.007699169000002115), (2, 0.01422596300000123), (3, 0.025658181999997254), (4, 0.05251848199999998), (8, 0.7712339829999983), (16, 15.596257020000003), (12, 16.206363621999998), (10, 3.314060688000005), (11, 7.543844796999998)]


 12%|██████████████▉                                                                                                        | 2/16 [01:00<05:57, 25.52s/it]

(approx_mc2_core_wrapper pid=32817) time_cost_info: [(1, 0.0075654319999998165), (2, 0.013557699000003254), (3, 0.02736868200000231), (4, 0.05352841399999875), (8, 0.9584883309999981), (16, 15.940654163000005), (12, 16.339536960000004), (10, 3.767446092), (11, 8.67028800100001)]


 25%|█████████████████████████████▊                                                                                         | 4/16 [01:01<01:43,  8.63s/it]

...

(approx_mc2_core_wrapper pid=32813) time_cost_info: [(1, 0.00782438800001728), (2, 0.012214198000037868), (3, 0.026208676999999625), (4, 0.04569378100001131), (8, 0.39361948399999847), (6, 0.13731225600002972), (7, 0.3784760389999633)]
(approx_mc2_core_wrapper pid=32814) time_cost_info: [(1, 0.007492011000010734), (2, 0.010691049999991264), (3, 0.021217811999974856), (4, 0.04929472000003443), (8, 0.3800823749999722), (6, 0.1154090320000023), (7, 0.24464415500000314)]
(approx_mc2_core_wrapper pid=32818) time_cost_info: [(1, 0.007639824000023054), (2, 0.013009878999980629), (3, 0.02442323299999316), (4, 0.04645231699998931), (8, 0.38148075499998413), (6, 0.12239749999997684), (7, 0.18270970600002556)]
(approx_mc2_core_wrapper pid=32827) time_cost_info: [(1, 0.006621386000006169), (2, 0.013807947000032073), (3, 0.021730655999988358), (4, 0.04575749700001097), (8, 0.3887245500000063), (6, 0.13413358200000403), (7, 0.23275328799996942)]
(approx_mc2_core_wrapper pid=32823) time_cost_info: [(

 69%|█████████████████████████████████████████████████████████████████████████████████▏                                    | 11/16 [00:00<00:00, 19.20it/s]

(approx_mc2_core_wrapper pid=32817) time_cost_info: [(8, 0.40081110399995623), (7, 0.2585968649999586)]
(approx_mc2_core_wrapper pid=32828) time_cost_info: [(8, 0.3770990030000121), (7, 0.2945366060000083)]
(approx_mc2_core_wrapper pid=32823) time_cost_info: [(8, 0.3874374890000354), (7, 0.24036362599997574)]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 16.10it/s]


(approx_mc2_core_wrapper pid=32815) time_cost_info: [(8, 0.37872988699996313), (7, 0.23631994999999506)]
(approx_mc2_core_wrapper pid=32816) time_cost_info: [(7, 0.24624931200003175), (8, 0.36177847199996904)]
(approx_mc2_core_wrapper pid=32813) time_cost_info: [(8, 0.39232872299999144), (7, 0.21395361899999443)]
(approx_mc2_core_wrapper pid=32814) time_cost_info: [(8, 0.3885248550000142), (7, 0.24074405400000387)]
(approx_mc2_core_wrapper pid=32818) time_cost_info: [(7, 0.26396345000000565), (8, 0.3507387729999891)]
(approx_mc2_core_wrapper pid=32827) time_cost_info: [(8, 0.39139235400000416), (7, 0.2355169530000012)]
(approx_mc2_core_wrapper pid=32820) time_cost_info: [(8, 0.38714186600003586), (7, 0.23777020800002902)]
(approx_mc2_core_wrapper pid=32819) time_cost_info: [(8, 0.396714118000034), (7, 0.2637025669999957)]
(approx_mc2_core_wrapper pid=32826) time_cost_info: [(8, 0.388729737999995), (7, 0.20317093100004513)]
(approx_mc2_core_wrapper pid=32825) time_cost_info: [(8, 0.3990

In [6]:
df = pd.DataFrame(
    res_rows,
    columns=['num_rules', 'running_time_approx_mc2', 'running_time_bb', 'estimate_count', 'true_count']
)
df['runtime-factor'] = df['running_time_approx_mc2'] / df['running_time_bb']
df['estimation-rel-diff'] = (df['estimate_count'] - df['true_count']) / df['true_count']

df



,num_rules,running_time_approx_mc2,running_time_bb,estimate_count,true_count,runtime-factor,estimation-rel-diff
0,200,107.465775,14.017417,159744,164189,7.666589,-0.027072
1,150,74.586741,7.343284,319488,318240,10.157137,0.003922
2,100,54.220374,4.869121,169984,166046,11.135557,0.023716
3,50,2.274073,0.227911,12544,11797,9.977881,0.063321


In [7]:
print(df.groupby('num_rules')['running_time_approx_mc2', 'running_time_bb', 'runtime-factor'].mean().to_markdown())

|   num_rules |   running_time_approx_mc2 |   running_time_bb |   runtime-factor |
|------------:|--------------------------:|------------------:|-----------------:|
|          50 |                   2.27407 |          0.227911 |          9.97788 |
|         100 |                  54.2204  |          4.86912  |         11.1356  |
|         150 |                  74.5867  |          7.34328  |         10.1571  |
|         200 |                 107.466   |         14.0174   |          7.66659 |


/var/folders/56/m2bm9k3508q4_2f94y87db9c0000gv/T/ipykernel_32795/3520125498.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  print(df.groupby('num_rules')['running_time_approx_mc2', 'running_time_bb', 'runtime-factor'].mean().to_markdown())
